## Test Property Filters

In [1]:

from Bio.Seq import Seq
from Bio.SeqUtils import MeltingTemp as mt

from oligo_designer_toolsuite.oligo_property_filter import (
    GCContent,
    MaskedSequences,
    MeltingTemperatureNN,
)
from oligo_designer_toolsuite.oligo_property_filter import (
    PadlockArms,
)
from oligo_designer_toolsuite.oligo_property_filter import (
    PropertyFilter,
)

In [2]:
## Melting temp parameters


Tm_parameters = {
    "check": True,
    "strict": True,
    "c_seq": None,
    "shift": 0,
    "nn_table": getattr(mt, "DNA_NN3"),
    "tmm_table": getattr(mt, "DNA_TMM1"),
    "imm_table": getattr(mt, "DNA_IMM1"),
    "de_table": getattr(mt, "DNA_DE1"),
    "dnac1": 50,  # [nM]
    "dnac2": 0,
    "selfcomp": False,
    "dNTPs": 0,
    "saltcorr": 7,
    "Na": 1.25,  # [mM]
    "K": 75,  # [mM]
    "Tris": 20,  # [mM]
    "Mg": 10,  # [mM]
}

Tm_chem_correction_parameters = {
    "DMSO": 0,
    "DMSOfactor": 0.75,
    "fmdfactor": 0.65,
    "fmdmethod": 1,
    "GC": None,
    "fmd": 20,
}

### Test masked Sequences

In [3]:
masked_sequences_filter = MaskedSequences(mask="N")

seq_remove = Seq("TGTCGGATCTCNTCAACAAGCTGGTCNTGA")
res, _ = masked_sequences_filter.apply(seq_remove)
assert (
    res == False
), f"error: A sequence ({seq_remove}) not fulfilling the mask condition has been accepted!"

seq_keep = Seq("TGTCGGATCTCTTCAACAAGCTGGTCATGA")
res, _ = masked_sequences_filter.apply(seq_keep)
assert (
    res == True
), f"error: A sequence ({seq_keep}) fulfilling the mask conditions has not been accepted!"

### Test GC content filter

In [4]:
GC_content_filter = GCContent(GC_content_min=40, GC_content_max=60)

seq_remove = Seq("TCGGGCGGGAGATCCAGGTGGCGCGCAAAG")
res, _ = GC_content_filter.apply(seq_remove)
assert (
    res == False
), f"error: A sequence ({seq_remove}) not fulfilling the GC content condition has been accepted!"

seq_keep = Seq("TGTCGGATCTCTTCAACAAGCTGGTCATGA")
res, _ = GC_content_filter.apply(seq_keep)
assert (
    res == True
), f"error: A sequence ({seq_keep}) fulfilling the GC content conditions has not been accepted!"

### Test melting Temperature Filter

In [5]:
# Test if Tm filter works with default parameters
Tm_filter = MeltingTemperatureNN(Tm_min=52, Tm_max=67, Tm_parameters={})

seq_remove = Seq("TGGCTTGGGCCTTTCCAAGCCCCCATTTGAGCT")
res, _ = Tm_filter.apply(seq_remove)
assert (
    res == False
), f"error: A sequence ({seq_remove}) not fulfilling the Tm condition with has been accepted!"

seq_keep = Seq("TGTCGGATCTCTTCAACAAGCTGGTCATGA")
res, _ = Tm_filter.apply(seq_keep)
assert (
    res == True
), f"error: A sequence ({seq_keep}) fulfilling the Tm conditions has not been accepted!"

# Test if Tm filter works with user-defined Tm parameters
Tm_filter = MeltingTemperatureNN(
    Tm_min=52,
    Tm_max=67,
    Tm_parameters=Tm_parameters,
    Tm_chem_correction_parameters=Tm_chem_correction_parameters,
)

seq_remove = Seq("TGGCTTGGGCCTTTCCAAGCCCCCATTTGAGCT")
res, _ = Tm_filter.apply(seq_remove)
assert (
    res == False
), f"error: A sequence ({seq_remove}) not fulfilling the Tm condition with user-defined parameters has been accepted!"

seq_keep = Seq("TGTCGGATCTCTTCAACAAGCTGGTCATGA")
res, _ = Tm_filter.apply(seq_keep)
assert (
    res == True
), f"error: A sequence ({seq_keep}) fulfilling the Tm conditions with user-defined parameters has not been accepted!"